In [1]:
import struct
from dataclasses import dataclass
from typing import List, Tuple, Union

import numpy as np

In [2]:
tmp = np.load("../test_messung_burstCnt_1.npz", allow_pickle=True)
tmp.files

['data_hex', 'data_int']

In [317]:
def del_hex_in_list(lst: list) -> list:
    return [
        "0" + ele.replace("0x", "") if len(ele) == 1 else ele.replace("0x", "")
        for ele in lst
    ]


def hex_to_float(input_list: list, exp: int = 0) -> float:
    return int("".join(input_list), 16) * 10 ** (exp)


def reshape_measurement_buffer(lst: list) -> np.ndarray:
    idx_b4 = [i for i, ele in enumerate(lst) if ele == "b4"]
    idx_b4 = np.array(idx_b4)
    step = abs(idx_b4[1] - idx_b4[0]) + 1
    return np.array(
        [lst[i : i + step] for i in range(idx_b4[0], idx_b4[-1] + step, step)],
        dtype=list,
    )


@dataclass
class SingleFrame:
    start_tag: List[str]
    channel_group: str
    excitation_stgs: List[str]
    frequency_row: List[str]
    timestamp: List[str]
    ch_1: complex
    ch_2: complex
    ch_3: complex
    ch_4: complex
    ch_5: complex
    ch_6: complex
    ch_7: complex
    ch_8: complex
    ch_9: complex
    ch_10: complex
    ch_11: complex
    ch_12: complex
    ch_13: complex
    ch_14: complex
    ch_15: complex
    ch_16: complex
    end_tag: str


def parse_single_frame(lst_ele: np.ndarray) -> SingleFrame:
    channels = {}
    enum = 0
    for i in range(11, 135, 8):
        enum += 1
        channels[f"ch_{enum}"] = complex(
            hex_to_float(lst_ele[i : i + 4], exp=-9),
            hex_to_float(lst_ele[i + 4 : i + 8], exp=-9),
        )

    excitation_stgs = np.array([int(ele) for ele in lst_ele[3:5]])

    sgl_frm = SingleFrame(
        start_tag=lst_ele[0],
        channel_group=int(lst_ele[2]),
        excitation_stgs=excitation_stgs,
        frequency_row=lst_ele[5:7],
        timestamp=lst_ele[7:11],
        **channels,
        end_tag=lst_ele[139],
    )
    return sgl_frm

In [318]:
lst = del_hex_in_list(tmp["data_hex"])

In [319]:
l = reshape_measurement_buffer(lst)

In [320]:
l[:5]

array([list(['b4', '89', '1', '1', '2', '0', '0', '0', '0', '0', '0', '3f', '9c', 'cc', '75', 'be', 'ad', 'd1', 'd9', 'bf', '9c', 'a9', 'f3', '3e', 'af', '61', '1a', 'bd', '8a', '20', 'ac', 'bd', '7a', 'ae', 'd7', 'bc', '2', '25', 'c', 'bc', '19', '46', '19', '3a', '90', '20', 'ab', '39', '9a', 'de', '4e', '39', '15', 'c1', 'fa', 'b8', '55', '81', '34', '3a', '9', '40', 'b8', '39', 'd7', 'fe', 'ce', 'b6', 'ab', '13', '67', 'b9', '8', 'd7', 'dd', 'ba', '26', 'c9', '4e', 'ba', '4a', '19', '55', '3a', '46', '51', '66', '3a', '32', 'd7', 'ab', 'b9', 'c8', '89', '1c', 'ba', '18', 'b7', 'd2', '3a', '32', '2b', '4', '39', 'fb', '1f', '57', 'ba', '6b', 'b0', '6f', 'ba', '83', 'd8', '1c', '37', 'fb', 'fc', '56', 'b8', '80', 'f4', '65', '3a', '87', 'c0', '1a', '3a', '87', 'e2', '2d', '3a', 'fb', '34', 'a1', '3a', 'f7', 'ce', '93', 'b4']),
       list(['b4', '89', '2', '1', '2', '0', '0', '0', '0', '0', '0', 'bc', '65', '91', 'c1', 'bb', 'e2', '11', '6b', 'bb', '60', 'd0', 'f2', 'bb', '13', '82',

In [321]:
sf = parse_single_frame(l[10])

In [322]:
sf.__dict__

{'start_tag': 'b4',
 'channel_group': 3,
 'excitation_stgs': array([3, 4]),
 'frequency_row': ['0', '0'],
 'timestamp': ['0', '0', '0', '1f'],
 'ch_1': (3.164244375+3.16724153j),
 'ch_2': (0.9883375660000001+0.98383458j),
 'ch_3': (3.0534327250000004+3.0528875180000004j),
 'ch_4': (3.0534327250000004+3.0528875180000004j),
 'ch_5': (3.0534327250000004+3.0528875180000004j),
 'ch_6': (3.0534327250000004+3.0528875180000004j),
 'ch_7': (3.0534327250000004+3.0528875180000004j),
 'ch_8': (3.0534327250000004+3.0528875180000004j),
 'ch_9': (3.0534327250000004+3.0528875180000004j),
 'ch_10': (3.0534327250000004+3.0528875180000004j),
 'ch_11': (3.0534327250000004+3.0528875180000004j),
 'ch_12': (3.0534327250000004+3.0528875180000004j),
 'ch_13': (3.0534327250000004+3.0528875180000004j),
 'ch_14': (3.0534327250000004+3.0528875180000004j),
 'ch_15': (3.0534327250000004+3.0528875180000004j),
 'ch_16': (3.0534327250000004+3.0528875180000004j),
 'end_tag': 'b4'}

In [323]:
l

array([list(['b4', '89', '1', '1', '2', '0', '0', '0', '0', '0', '0', '3f', '9c', 'cc', '75', 'be', 'ad', 'd1', 'd9', 'bf', '9c', 'a9', 'f3', '3e', 'af', '61', '1a', 'bd', '8a', '20', 'ac', 'bd', '7a', 'ae', 'd7', 'bc', '2', '25', 'c', 'bc', '19', '46', '19', '3a', '90', '20', 'ab', '39', '9a', 'de', '4e', '39', '15', 'c1', 'fa', 'b8', '55', '81', '34', '3a', '9', '40', 'b8', '39', 'd7', 'fe', 'ce', 'b6', 'ab', '13', '67', 'b9', '8', 'd7', 'dd', 'ba', '26', 'c9', '4e', 'ba', '4a', '19', '55', '3a', '46', '51', '66', '3a', '32', 'd7', 'ab', 'b9', 'c8', '89', '1c', 'ba', '18', 'b7', 'd2', '3a', '32', '2b', '4', '39', 'fb', '1f', '57', 'ba', '6b', 'b0', '6f', 'ba', '83', 'd8', '1c', '37', 'fb', 'fc', '56', 'b8', '80', 'f4', '65', '3a', '87', 'c0', '1a', '3a', '87', 'e2', '2d', '3a', 'fb', '34', 'a1', '3a', 'f7', 'ce', '93', 'b4']),
       list(['b4', '89', '2', '1', '2', '0', '0', '0', '0', '0', '0', 'bc', '65', '91', 'c1', 'bb', 'e2', '11', '6b', 'bb', '60', 'd0', 'f2', 'bb', '13', '82',

In [309]:
len(l)

129

In [316]:
l[128]

[]